

## หัวข้อ: การแยก Named Entities ด้วย OpenAI API

---

## 🎯 วัตถุประสงค์

ให้เขียน prompt สำหรับใช้งานร่วมกับ **OpenAI API** เพื่อแยก **Named Entities** จากประโยคภาษาไทย พร้อมจัดหมวดหมู่ให้ถูกต้องตามที่กำหนด และแสดงผลลัพธ์ในรูปแบบ **JSON dictionary**

> ❗ **ห้าม** มีคำอธิบายเพิ่มเติมอื่นใดนอกจาก JSON เท่านั้น

---

## ✅ หมวดหมู่ของ Named Entities ที่ต้องใช้

| หมวดหมู่          | ความหมาย                            |
| ----------------- | ----------------------------------- |
| 👤 บุคคล          | ชื่อบุคคล                           |
| 🏛️ องค์กร        | หน่วยงาน บริษัท รัฐวิสาหกิจ         |
| 📍 สถานที่        | จังหวัด เมือง ประเทศ                |
| 🗓️ วันที่ / เวลา | วัน เดือน ปี เวลา                   |
| 🔢 จำนวน          | ตัวเลขที่มีหน่วย เช่น กรัม กิโลเมตร |
| 💰 จำนวนเงิน      | ตัวเลขพร้อมสกุลเงิน                 |
| 📊 เปอร์เซ็นต์    | ค่าร้อยละ เช่น 75%                  |
| 📚 ผลงาน          | ชื่อหนังสือ เพลง ภาพยนตร์ ฯลฯ       |
| 📦 สินค้า         | สินค้าหรือผลิตภัณฑ์                 |
| 🈯 ภาษา           | ภาษาที่กล่าวถึง เช่น ภาษาไทย        |

---

## 🧪 ตัวอย่าง Prompt & Output

### 🔹 ตัวอย่างที่ 1

**ประโยค:**
`สมเด็จพระนางเจ้าสิริกิติ์ เสด็จเยือนกรุงเทพมหานครเมื่อวันที่ 1 มิถุนายน 2025`

**ผลลัพธ์ (JSON):**

```json
{
  "บุคคล": "สมเด็จพระนางเจ้าสิริกิติ์",
  "สถานที่": "กรุงเทพมหานคร",
  "วันที่": "1 มิถุนายน 2025"
}
```

---

### 🔹 ตัวอย่างที่ 2

**ประโยค:**
`AIS เปิดตัวโปรโมชั่นใหม่สำหรับ iPhone 12`

**ผลลัพธ์ (JSON):**

```json
{
  "องค์กร": "AIS",
  "สินค้า": "iPhone 12"
}
```

---



In [ ]:
import IPython
import sys

def clean_notebook():
    IPython.display.clear_output(wait=True)
    print("Notebook cleaned.")

!pip install openai
!pip install gradio
!pip install python-dotenv

# Clean up the notebook
clean_notebook()

Notebook cleaned.


In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

True

In [2]:
import os
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model_name = "gpt-4.1"    

In [ ]:
import os
from openai import OpenAI
import json

# กำหนด API Key
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# โมเดลที่ใช้
model_name = "gpt-4.1"  # หรือ "gpt-3.5-turbo" ก็ได้




ner_questions = [
"สมเด็จพระนางเจ้าสิริกิติ์ เสด็จเยือนกรุงเทพมหานครเมื่อวันที่ 1 มิถุนายน 2025",
"AIS เปิดตัวโปรโมชั่นใหม่สำหรับ iPhone 12",
"ภาพยนตร์เรื่อง เศรษฐศาสตร์พอเพียง ได้รับรางวัลระดับนานาชาติ",
"บริษัท ปตท. จำกัด (มหาชน) มียอดขายเพิ่มขึ้น 75%",
"ฉันซื้อแอปเปิ้ลน้ำหนัก 500 กรัม ในราคา 1,000 บาท",
"นักเรียนไทยควรเรียนรู้ภาษาอังกฤษและภาษาไทยควบคู่กันไป",
"วันที่ 10 ตุลาคม 2025 จะมีงานเทศกาลที่จังหวัดเชียงใหม่",
"ดร.สมชาย เป็นผู้เขียนหนังสือเกี่ยวกับปัญญาประดิษฐ์",
"Apple เปิดตัวผลิตภัณฑ์ใหม่ชื่อ MacBook Pro",
"เพลย์ลิสต์เพลงของเขามีแต่เพลงจากวง Bodyslam",
"Google ประกาศตั้งสำนักงานแห่งใหม่ในประเทศสิงคโปร์",
"ราคาทองคำวันนี้อยู่ที่ 32,500 บาทต่อบาททองคำ",
"บริษัท Tesla มีมูลค่าตลาดเพิ่มขึ้น 10%",
"งานสัปดาห์หนังสือแห่งชาติจะจัดขึ้นที่กรุงเทพฯ",
"ภาษาอังกฤษเป็นภาษาสากลที่ใช้ในหลายประเทศ",
"หนังสือเรื่อง 'ปรัชญาชีวิต' แต่งโดย อาจารย์ไพฑูรย์",
"Samsung Galaxy S24 มีกล้องที่มีความละเอียดสูงถึง 200 ล้านพิกเซล",
"ในวันที่ 14 กุมภาพันธ์ ร้านค้าส่วนใหญ่มีโปรโมชั่นลดราคา",
"ผลิตภัณฑ์จากญี่ปุ่นได้รับความนิยมอย่างมากในประเทศไทย",
"นักกีฬาชื่อดังจากประเทศฝรั่งเศสมาเยือนกรุงเทพฯ"
]


for index, question in enumerate(ner_questions):
result = extract_entities(question)
print(f"Question {index + 1}: {question}")
print(f"Result: {result}\n")    

{'องค์กร': 'AIS', 'สินค้า': 'iPhone 12'}
Question 1: สมเด็จพระนางเจ้าสิริกิติ์ เสด็จเยือนกรุงเทพมหานครเมื่อวันที่ 1 มิถุนายน 2025
Result: {'บุคคล': 'สมเด็จพระนางเจ้าสิริกิติ์', 'สถานที่': 'กรุงเทพมหานคร', 'วันที่': '1 มิถุนายน 2025'}

Question 2: AIS เปิดตัวโปรโมชั่นใหม่สำหรับ iPhone 12
Result: {'องค์กร': 'AIS', 'สินค้า': 'iPhone 12'}

Question 3: ภาพยนตร์เรื่อง เศรษฐศาสตร์พอเพียง ได้รับรางวัลระดับนานาชาติ
Result: {'ผลงาน': 'เศรษฐศาสตร์พอเพียง'}

Question 4: บริษัท ปตท. จำกัด (มหาชน) มียอดขายเพิ่มขึ้น 75%
Result: {'องค์กร': 'บริษัท ปตท. จำกัด (มหาชน)', 'เปอร์เซ็นต์': '75%'}

Question 5: ฉันซื้อแอปเปิ้ลน้ำหนัก 500 กรัม ในราคา 1,000 บาท
Result: {'สินค้า': 'แอปเปิ้ล', 'จำนวน': '500 กรัม', 'จำนวนเงิน': '1,000 บาท'}

Question 6: นักเรียนไทยควรเรียนรู้ภาษาอังกฤษและภาษาไทยควบคู่กันไป
Result: {'บุคคล': 'นักเรียนไทย', 'ภาษา': ['ภาษาอังกฤษ', 'ภาษาไทย']}

Question 7: วันที่ 10 ตุลาคม 2025 จะมีงานเทศกาลที่จังหวัดเชียงใหม่
Result: {'วันที่': '10 ตุลาคม 2025', 'สถานที่': 'จังหวัดเชียงใหม่'}

Questio